<h1>Выбор локации для скважины<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели-для-каждого-региона." data-toc-modified-id="Обучение-и-проверка-модели-для-каждого-региона.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели для каждого региона.</a></span></li><li><span><a href="#Подготовка-данных-к-рассчету-прибыли" data-toc-modified-id="Подготовка-данных-к-рассчету-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка данных к рассчету прибыли</a></span></li><li><span><a href="#Реализация-функции-для-расчета-прибыли" data-toc-modified-id="Реализация-функции-для-расчета-прибыли-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Реализация функции для расчета прибыли</a></span></li><li><span><a href="#Расчет-прибыли-и-рисков" data-toc-modified-id="Расчет-прибыли-и-рисков-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Расчет прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#example-5" data-toc-modified-id="example-5-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>example 5</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

**Введение**

Для исследования предоставлены данные проб нефти из трех регионов:

* `/datasets/geo_data_0.csv`


* `/datasets/geo_data_1.csv`


* `/datasets/geo_data_2.csv`
    
    

**Цель исследования**

Построить модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализирировать возможную прибыль и риски.

**Ход исследования**

1. Загрузить и подготовить данные. 


2. Обучить и проверить модель для каждого региона. 


3. Подготовить данные к рассчету прибыли.


4. Напиcать функцию для расчёта прибыли. 


5. Посчитать риски и прибыль для каждого региона. 


6. Написать вывод. 

**Информация о стоблцах** 

* `id` — уникальный идентификатор скважины;


* `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);


* `product` — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных 

Подключим все необходимые библиотеки.

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy import stats as st
from numpy.random import RandomState
state = RandomState(12) 

Выгрузим данные в переменные:
    
* data_1 – `/datasets/geo_data_0.csv`


* data_2 – `/datasets/geo_data_1.csv`


* data_3 – `/datasets/geo_data_2.csv`

In [2]:
pth_1 = '/datasets/geo_data_0.csv'
pth_2 = '/datasets/geo_data_1.csv'
pth_3 = '/datasets/geo_data_2.csv'

pth_1_1 = 'geo_data_0.csv'
pth_2_1 = 'geo_data_1.csv'
pth_3_1 = 'geo_data_2.csv'

if ( os.path.exists(pth_1) and os.path.exists(pth_2) and os.path.exists(pth_3) ):
    data_1 = pd.read_csv('/datasets/geo_data_0.csv')
    data_2 = pd.read_csv('/datasets/geo_data_1.csv') 
    data_3 = pd.read_csv('/datasets/geo_data_2.csv')

else:
    data_1 = pd.read_csv('geo_data_0.csv')
    data_2 = pd.read_csv('geo_data_1.csv') 
    data_3 = pd.read_csv('geo_data_2.csv')

In [3]:
def head_and_info(data):
    display(data.head(5))
    data.info()
    print("\n-----------------------------------------")

In [4]:
head_and_info(data_1)
head_and_info(data_2)
head_and_info(data_3)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

-----------------------------------------


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

-----------------------------------------


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

-----------------------------------------


In [5]:
display(data_2.head(5))

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
display(data_3.head(5))

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


**Промежуточный вывод:** 


* Данные выгружены корректно.

* Типы данных – коректны. 

* Пропуски – отсутствуют.

* Столбцы приведены к нижнему регистру. 

**Промежуточный вывод**: данные выгружены корректно.

Проверим данные на наличие дубликатов.

In [7]:
for data, n in zip([data_1, data_2, data_3], range(1,4)):
    print(f'Количество дубликатов в data_{n}: {data.duplicated().sum()}')

Количество дубликатов в data_1: 0
Количество дубликатов в data_2: 0
Количество дубликатов в data_3: 0


Выведем информацию о данных.

In [8]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Проверим столбец `id` на уникальность.  

In [11]:
display(data_1['id'].unique)
display(data_2['id'].unique)
display(data_3['id'].unique)

<bound method Series.unique of 0        txEyH
1        2acmU
2        409Wp
3        iJLyR
4        Xdl7t
         ...  
99995    DLsed
99996    QKivN
99997    3rnvd
99998    7kl59
99999    1CWhH
Name: id, Length: 100000, dtype: object>

<bound method Series.unique of 0        kBEdx
1        62mP7
2        vyE1P
3        KcrkZ
4        AHL4O
         ...  
99995    QywKC
99996    ptvty
99997    09gWa
99998    rqwUm
99999    relB0
Name: id, Length: 100000, dtype: object>

<bound method Series.unique of 0        fwXo0
1        WJtFt
2        ovLUW
3        q6cA6
4        WPMUX
         ...  
99995    4GxBu
99996    YKFjq
99997    tKPY3
99998    nmxp2
99999    V9kWn
Name: id, Length: 100000, dtype: object>

**Промежуточный вывод:**

Дубликаты и повторения отсутсвуют. 

**Подготовка данных** 

Выведем данные.

In [12]:
display(data_1.head(5))
display(data_2.head(5))
display(data_3.head(5))

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Видно, что необходимо избавиться от столбца `id`, так как при обучении он будет сбивать модель.

In [13]:
data_1 = data_1.drop('id', axis = 1)
data_2 = data_2.drop('id', axis = 1)
data_3 = data_3.drop('id', axis = 1)

Проверим признаки на наличие мультиколлинеарность.

In [14]:
display(data_1.corr())
display(data_2.corr())
display(data_3.corr())

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Мультиколлинеарность –  `отсутствует `. 

**Общий вывод:**

В ходе обработки и загрузки данных, были выполнены следующие пукты: 

* Корректно выгружены все необходимые данные.

* Проверены тип, дубликаты, регистр и пропуски в столбцах. 

* Удалены неинформативные столбцы. 

* Зафиксирована уникальность столбца `id`.

* Утсановлен факт остутсвия  `мультиколлинеарности`

## Обучение и проверка модели для каждого региона. 

Разделим данные на `обучающую` и `валидационную` выборки.

In [15]:
data_1_train, 
data_1_valid, 
data_2_train, 
data_2_valid, 
data_3_train, 
data_3_valid = train_test_split(data_1, 
                                data_2, 
                                data_3, 
                                test_size=0.25, 
                                random_state=state)


features_1_train = data_1_train.drop('product', axis = 1)
target_1_train = data_1_train['product']
features_1_valid = data_1_valid.drop('product', axis = 1)
target_1_valid = data_1_valid['product']

features_2_train = data_2_train.drop('product', axis = 1)
target_2_train = data_2_train['product']
features_2_valid = data_2_valid.drop('product', axis = 1)
target_2_valid = data_2_valid['product']

features_3_train = data_3_train.drop('product', axis = 1)
target_3_train = data_3_train['product']
features_3_valid = data_3_valid.drop('product', axis = 1)
target_3_valid = data_3_valid['product']

Обучим модель **LinearRegression** и сохраним предсказания для каждого региона.  

In [16]:
model = LinearRegression()
    
model.fit(features_1_train, target_1_train)
predictions_1 = model.predict(features_1_valid)

model.fit(features_2_train, target_2_train)
predictions_2 = model.predict(features_2_valid)

model.fit(features_3_train, target_3_train)
predictions_3 = model.predict(features_3_valid)

Найдем средний запас предсказанного сырья и RMSE модели для кадого региона.

**RMSE**

In [17]:
print(f'RMSE for data_1: {mean_squared_error(target_1_valid,predictions_1)**0.5}')
print(f'RMSE for data_2: {mean_squared_error(target_2_valid,predictions_2)**0.5}')
print(f'RMSE for data_3: {mean_squared_error(target_3_valid ,predictions_3)**0.5}')

RMSE for data_1: 37.78806596603057
RMSE for data_2: 0.8885339556138786
RMSE for data_3: 39.800203117903365


**Средний запас предсказанного сырья**

In [18]:
print(f'data_1: {predictions_1.mean()}')
print(f'data_2: {predictions_2.mean()}')
print(f'data_3: {predictions_3.mean()}')

data_1: 92.13368147346338
data_2: 69.0378932957214
data_3: 95.05096092338655


**Общий вывод:** 

В ходе обучения были получены значение `RMSE` для каждого региона: 

* RMSE for data_1: 37.5794217150813

* RMSE for data_2: 0.8930992867756166

* RMSE for data_3: 40.02970873393434

Видно, что *средняя квадратическая ошибка* у региона `data_1` и `data_3` почти одинаковые, а у `data_2` меньше единицы, что говорит о почти полностью точных предсказаниях модели.


Также, был рассчитан *средний запас предсказанного сырья*:

* data_1: 92.59256778438035
* data_2: 68.728546895446
* data_3: 94.96504596800492

Видно, что в регионах `data_1` и `data_2` намного больше запасов сырья чем в `data_2`. 

## Подготовка данных к рассчету прибыли

Подготовим все необходимые переменные для рассчета прибыли:

* budget = 10 млд. 

* sump(необходимое количество скважин) = 200

* cost_per_bar = 450


* product - ?

In [19]:
BUDJET = 10_000_000_000
INCOME_PER_BARREL = 450_000
SUMP = 200

Рассчитаем достаточный объем сырья для безубыточной разработки новой скважины.

In [20]:
PRODUCT = round(BUDJET / (SUMP * INCOME_PER_BARREL))
print(f'{PRODUCT} тыс.')

111 тыс.


Следовательно, для безубыточной работы скважины необходимо, чтобы она приносила `111` тыс. бар. 

Сравним полученный объем сырья со средним запасом в каждом регионе.

In [21]:
print(data_1['product'].mean())
print(data_2['product'].mean())
print(data_3['product'].mean())

92.50000000000001
68.82500000000002
95.00000000000004


**Общий вывод:**

В ходе работы была найдена необходимая велична для безубыточной работы скважины – `111` тыс. бар.

Средний запас в каждом регионе: 

* data_1 = 92.49999999999974

* data_2 = 68.82500000002561

* data_3 = 95.00000000000041

Видно, что средний запас сырья в каждом регионе – меньше необходимого для безубыточной работы.

## Реализация функции для расчета прибыли

Напишем функцию для реализации расчета прибыли.

In [22]:
predictions_1 = pd.Series(predictions_1)
predictions_2 = pd.Series(predictions_2)
predictions_3 = pd.Series(predictions_3)

target_1_valid = target_1_valid.reset_index(drop = True)
target_2_valid = target_2_valid.reset_index(drop = True)
target_3_valid = target_3_valid.reset_index(drop = True)

In [23]:
def profit(predictions, target, SUMP):
    top_preds = predictions.sort_values(ascending=False)
    top_target = target[top_preds.index][:SUMP]
    revenue = top_target.sum() * INCOME_PER_BARREL
    return revenue - BUDJET

Воспользуемся фунцией для каждого региона. 

In [24]:
for preds, target, i in zip([predictions_1, predictions_2, predictions_3], 
                            [target_1_valid, target_2_valid, target_3_valid],
                            [1,2,3]):
    print(f"Region {i}:")
    print("\n-----------------------------------------")
    print(f'Mean: {profit(preds, target, SUMP)}')
    print("-----------------------------------------\n\n")

Region 1:

-----------------------------------------
Mean: 3418631960.7177105
-----------------------------------------


Region 2:

-----------------------------------------
Mean: 2415086696.681511
-----------------------------------------


Region 3:

-----------------------------------------
Mean: 2549949202.100603
-----------------------------------------




## Расчет прибыли и рисков

### example 5

Посчитав риски и прибыль для каждого региона, применив технику `Bootstrap` с `1000` выборок.

In [26]:
state = RandomState(12) 
def find_conf_interval_and_loss_risk(predictions, target):
    revenue = []
    for _ in range(1000):
        target_sample = target.sample(500, replace=True, random_state=state)
        predictions_sample = predictions[target_sample.index]
        revenue.append(profit(predictions_sample, target_sample, SUMP))
        
    lower = int(np.percentile(revenue, 2.5))
    higher = int(np.percentile(revenue, 97.5))
    mean_revenue = int(sum(revenue) / len(revenue))
    risk = st.percentileofscore(revenue, 0)

    return ((lower, higher), mean_revenue, risk)

In [27]:
for preds, target, i in zip([predictions_1, predictions_2, predictions_3], 
                            [target_1_valid, target_2_valid, target_3_valid],
                            [1,2,3]):
    
    interval, mean_revenue, risk = find_conf_interval_and_loss_risk(preds, target)
    print(f"Region {i}:")
    print("\n-----------------------------------------")
    print(f"Mean: {mean_revenue}")
    print(f"95% confidence interval: {interval}")
    print(f"Risk of loss: {risk}")
    print("-----------------------------------------\n\n")

Region 1:

-----------------------------------------
Mean: 475541876
95% confidence interval: (-50083021, 1008353695)
Risk of loss: 3.9000000000000004
-----------------------------------------


Region 2:

-----------------------------------------
Mean: 538007940
95% confidence interval: (133808296, 965199255)
Risk of loss: 0.30000000000000004
-----------------------------------------


Region 3:

-----------------------------------------
Mean: 370412660
95% confidence interval: (-152777383, 916584603)
Risk of loss: 9.0
-----------------------------------------




## Вывод

В ходе работы были выполнены следующие пункты: 


1. Загружены и подготовленны данные

    * Корректно выгружены все необходимые данные.

    * Проверены тип, дубликаты, регистр и пропуски в столбцах. 

    * Удалены неинформативные столбцы. 

    * Зафиксирована уникальность столбца `id`.

    * Утсановлен факт остутсвия  `мультиколлинеарности`

2. Обучена и проверена модель для каждого региона

    В ходе обучения были получены значение `RMSE` для каждого региона: 

    * RMSE for data_1: 37.5794217150813

    * RMSE for data_2: 0.8930992867756166

    * RMSE for data_3: 40.02970873393434

    Видно, что *средняя квадратическая ошибка* у региона `data_1` и `data_3` почти одинаковые, а у `data_2` меньше единицы,         что говорит о почти полностью точных предсказаниях модели.

    Также, был рассчитан *средний запас предсказанного сырья*:

   * data_1: 92.59256778438035 
    
   * data_2: 68.728546895446 
    
   * data_3: 94.96504596800492
    
   Видно, что в регионах `data_1` и `data_2` намного больше запасов сырья чем в `data_2`

3. Рассчитан достаточный объем сырья для безубыточной разработки новой скважины.

  * `111` тыс. бар.

4. Реализована функции для расчета прибыли

5. Рассчитана прибыль и риски для каждого региона. 

**Region 1:**

-----------------------------------------
Mean: 419930177;
95% confidence interval: (-141110488, 956814437);
Risk of loss: 7.5
-----------------------------------------


**Region 2:**

-----------------------------------------
Mean: 516594341;
95% confidence interval: (116606932, 930743658);
Risk of loss: 1.1
-----------------------------------------


**Region 3:**

-----------------------------------------
Mean: 407869169;
95% confidence interval: (-145646379, 986581470);
Risk of loss: 8.3
-----------------------------------------

Следовательно, из последнего пункта можно сделать итоговый вывод, что лучше всего подходит **`REGION 1`**, так как имеет минимальные риски и высокую среднюю прибыль, относительно других регионов.